<a href="https://colab.research.google.com/github/GoogleCloudPlatform/gcp-getting-started-lab-jp/blob/master/machine_learning/cloud_ai_building_blocks/speech_to_speech.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


--- 
```
Copyright 2019 Google LLC

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.
```
---

## Google Cloud API の認証情報を入力

Google Cloud API を REST インタフェースから利用するために、 [API Key](https://console.cloud.google.com/apis/credentials) を利用します。 Google Cloud Console から API Key をコピーしましょう。

In [0]:
import getpass
APIKEY = getpass.getpass()

## 音声データを録音する
この手順は Colaboratory でしか動作しません。 Colaboratory を使わない場合は別途 flac 形式の音声データを用意頂く必要があります。

In [0]:
#@title このセルを実行して record_audio を定義

# Install required libraries and packages
!pip install -qq pydub
!apt-get -qq update
!apt-get -qq install -y ffmpeg

# Define record_audio
import base64
import google.colab
import pydub

from io import BytesIO

def record_audio(file_id, framerate=16000, file_format='flac'):
  # Record webm file from Colaboratory.
  audio = google.colab._message.blocking_request(
    'user_media',
    {
      'audio': True,
      'video': False,
      'duration': -1
    },
    timeout_sec=600)

  # Convert web file into in_memory file.
  mfile = BytesIO(base64.b64decode(audio[audio.index(',')+1:]))

  # Store webm file locally.
  with open('{0}.webm'.format(file_id), 'wb') as f:
    mfile.seek(0)
    f.write(mfile.read())

  # Open stored web file and save it as wav with sample_rate=16000
  output_file = '{0}.{1}'.format(file_id, file_format)
  _ = pydub.AudioSegment.from_file('{0}.webm'.format(file_id), codec='opus')
  _.set_frame_rate(16000).export(output_file, format=file_format)

  return output_file

```record_audio``` を実行して音声を録音します。

In [0]:
record_audio('ja-sample')
audio_filename='ja-sample.flac'

録音した音声を確認します。

In [0]:
from IPython.display import Audio
Audio(audio_filename, rate=16000)

## Cloud Speech-to-Text API で音声データをテキスト化する

In [0]:
from googleapiclient.discovery import build
speech_service = build('speech', 'v1p1beta1', developerKey=APIKEY)

Cloud Speech-to-Text API に入力する情報を定義します.

In [0]:
from base64 import b64encode
from json import dumps

audio_filename = "ja-sample.flac" #@param {type:"string"}

with open(audio_filename, 'rb') as audio_file:
  content = b64encode(audio_file.read()).decode('utf-8')

my_audio = {
  'content': content
}

languageCode = 'ja-JP' #@param ["en-US", "ja-JP"]
model = 'default' #@param ["command_and_search", "phone_call", "video", "default"]

Cloud Speech-to-Text API の動作オプションを指定するための [RecognitionConfig](https://cloud.google.com/speech-to-text/docs/reference/rest/v1p1beta1/RecognitionConfig) を定義します。

In [0]:
my_recognition_config = {
  'encoding': 'FLAC',
  'sampleRateHertz': 16000,
  'languageCode': languageCode,
  'model': model
}

Cloud Speech-to-Text のリクエスト Body を定義します。

In [0]:
my_request_body={
  'audio': my_audio,
  'config': my_recognition_config,
}

Cloud Speech-to-Text にリクエストを送信します。

In [0]:
response = speech_service.speech().recognize(body=my_request_body).execute()

Cloud Speech-to-Text のレスポンスの中身を確認します。

In [0]:
import pprint
pprint.pprint(response)

レスポンスから日本語のテキストのみを抽出します。

In [0]:
ja_text = response['results'][0]['alternatives'][0]['transcript']

## Cloud Translation API で日本語テキストを英語テキストに翻訳する
Cloud Translation API では、数千もの言語ペアの間でダイナミックにテキストを翻訳できます。 Cloud Translation API を使えば、プログラム上でウェブサイトやアプリケーションを Google Translate と統合できます。

In [0]:
from googleapiclient.discovery import build
translate_service = build('translate', 'v2', developerKey=APIKEY)

Cloud Translation API に入力する情報を定義します。

In [0]:
source_language = "ja" #@param ["en", "ja"] {type: "string"}
target_language = "en" #@param ["en", "ja"] {type: "string"}

Cloud Translation API を実行します。

In [0]:
response = translate_service.translations().list(
  source=source_language,
  target=target_language,
  q=ja_text,
  model='nmt',
  format='html').execute()

Cloud Translation API のレスポンスの中身を確認します。

In [0]:
import pprint
pprint.pprint(response)

レスポンスから英語のテキストのみを抽出します。

In [0]:
en_text = response['translations'][0]['translatedText']

## Cloud Natural Language API で固有名詞を検出する

In [0]:
from googleapiclient.discovery import build
language_service = build('language', 'v1beta2', developerKey=APIKEY)

In [0]:
source_language = "en" #@param ["en", "ja"] {type: "string"}
document_type = "PLAIN_TEXT" #@param["PLAIN_TEXT", "HTML"] {type: "string"}
encoding_type = "UTF8" #@param["UTF8", "UTF16", "UTF32"] {type: "string"}

Cloud Natural Language API のエンティティ分析を実行します

In [0]:
response = language_service.documents().analyzeEntities(
  body={
    'document': {
      'type': document_type,
      'content': en_text,
      'language': source_language,
    },
    'encodingType': encoding_type,
  }
).execute()

Cloud Natural Language API のエンティティ分析のレスポンスを確認します。

In [0]:
import pprint
pprint.pprint(response)

レスポンスの中から固有名詞を抽出します。

In [0]:
proper_nouns = []

entities = response['entities']
for entity in entities:
  mentions = entity['mentions']
  for mention in mentions:
    if mention['type'] == 'PROPER':
      proper_nouns.append(mention['text']['content'])

print(proper_nouns)

## Cloud Text-to-Speech API で英語音声を合成する

In [0]:
from googleapiclient.discovery import build
texttospeech_service = build('texttospeech', 'v1beta1', developerKey=APIKEY)

### テキストから音声を合成する

[text.synthesize](https://cloud.google.com/text-to-speech/docs/reference/rest/v1beta1/text/synthesize) メソッドを使用すると、単語や文を自然な人間の音声の base64 でエンコードされた音声データに変換できます。このメソッドは、入力を生のテキストまたは[音声合成マークアップ言語（SSML）](https://cloud.google.com/text-to-speech/docs/ssml)として受け入れます。なお、合成する音声の種類は音声のサンプル付で[こちら](https://cloud.google.com/text-to-speech/docs/voices)に記載されています。

In [0]:
source_language = "en-US" #@param {type: "string"}
audio_encoding = 'OGG_OPUS' #@param ['OGG_OPUS', 'LINEAR16', 'MP3']
voice_gender = 'SSML_VOICE_GENDER_UNSPECIFIED' #@param ['FEMALE', 'MALE', 'NEUTRAL', 'SSML_VOICE_GENDER_UNSPECIFIED']
voice_name = 'en-US-Wavenet-D' #@param {type: "string"}

### 英語音声を合成する

In [0]:
response = texttospeech_service.text().synthesize(
  body={
    'input': {
      'text': en_text,
    },
    'voice': {
      'languageCode': source_language,
      'ssmlGender': voice_gender,
      'name': voice_name,
    },
    'audioConfig': {
      'audioEncoding': audio_encoding,
    },
  }
).execute()

In [0]:
import base64
from IPython.display import Audio

Audio(base64.b64decode(response['audioContent']))

### 音声マークアップを利用して固有名詞を強調した音声を合成する

In [0]:
en_ssml = """<speak>{0}</speak>""".format(en_text)
for proper_noun in proper_nouns:
  en_ssml = en_ssml.replace(
      proper_noun,
      '<prosody rate="slow" volume="loud">{0}</prosody>'.format(proper_noun))

print(en_ssml)

In [0]:
response = texttospeech_service.text().synthesize(
  body={
    'input': {
      'ssml': en_ssml,
    },
    'voice': {
      'languageCode': source_language,
      'ssmlGender': voice_gender,
      'name': voice_name,
    },
    'audioConfig': {
      'audioEncoding': audio_encoding,
    },
  }
).execute()

In [0]:
import base64
from IPython.display import Audio

Audio(base64.b64decode(response['audioContent']))